# Sensitivity/Specificity analysis demo

These are some examples of queries to directly interact with the API to get some metadata:

In [ ]:
from cdispyutils.hmac4 import get_auth
import json
import requests
with open('.secrets','r') as f:
    secrets = json.load(f)
auth = get_auth(secrets['access_key'], secrets['secret_key'], 'submission')

In [ ]:
print requests.get('http://api.internal.io/v0/submission', auth=auth).text
print requests.get('http://api.internal.io/v0/submission/bpa/', auth=auth).text

In [ ]:
data = {'query': """query Test { sample (project_id: "bpa-ThermoFisher_P0001_T1", submitter_id: "BPA-THERMOFISHER_P0001-S1") {   
                                 _aliquots_count aliquots { aliquot_concentration  _read_groups_count read_groups {
                                _submitted_somatic_mutations_count submitted_somatic_mutations {
                                file_name}} } }} """};
print requests.post('http://api.internal.io/v0/submission/graphql/', auth=auth, json=data).text

This example shows how to analyze data from one specific project in BloodPAC containing expected mutations from contrived samples and resulting VCF files

In [ ]:
import bpa_analysis_functions_v2 as bp

Get the project name we are going to analyze, local path to save project files and user profile to access their bucket:

In [ ]:
project = 'bpa-ThermoFisher_P0001_T1'
profile = 'bloodpac'
path    = 'files/'
bp.add_keys('.secrets')

Get project files from bucket to a local directory:

In [ ]:
bp.get_files_from_bucket(project, profile, path, '*.vcf')

List samples for this project:

In [ ]:
samples = bp.list_samples(project)
samples

List and count file types contained in this project

In [ ]:
vcf_files = bp.list_files_by_type(project,'VCF', samples[1])
vcf_files

In [ ]:
counts = bp.count_file_types(project, samples[1])
counts

Get expected mutations from sample-expectation metadata:

In [ ]:
expectation = bp.get_expected_mutations(project)
expectation

Calculate sensitivity and sensitivity values for one example VCF:

In [ ]:
sample = samples[1]
vcf_file = vcf_files[sample][2]
metrics = bp.calculate_metrics_vcf(project, path, vcf_file, sample)
metrics

Calculate sensitivity and sensitivity values for all VCFs in the project:

In [ ]:
table_metrics, data_metrics = bp.calculate_metrics_all_vcf(project, path, vcf_files)
table_metrics

In [ ]:
baseline_vcf = 'TFS.HZ.0.0.perc_20ng_TSVC_IonCodeTag_0101.vcf'
table_filter_metrics, data_filter_metrics = bp.calculate_metrics_all_vcf(project, path, vcf_files, baseline_vcf)
table_filter_metrics

In [ ]:
bp.plot_metrics(data_metrics, data_filter_metrics)